In [1]:
! pip install frictionless

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.9/309.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.4/90.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.7/411.7 kB 8.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 5.4 MB/s eta 0:00:00
  Created wheel for petl: filename=petl-1.7.14-py3-none-any.whl size=229615 sha256=8b664731cf53b98e3be9a48d04f3d3f2f6a6e1052767585f33f9cc02642056c2
  Stored in directory: /root/.cache/pip/wheels/65/d2/2c/655203aaf0c828a0b7481dfc3354814dd1b864f4c4b0a413a7
  Created wheel for stringcase: filename=stringcase-1.2.0-py3-none-any.whl size=3570 sh

In [2]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import time
import os
import frictionless
from getpass import getpass
from ftplib import FTP_TLS
from ftplib import FTP
import ftplib
import ast

In [3]:
import pandas as pd
import re
from urllib.request import Request, urlopen, urlretrieve
from bs4 import BeautifulSoup

url = 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/'

def extract_id(fileName):
    ogd_id = fileName.partition('_')[0]
    return ogd_id.replace('ogd', '')

def getInformation(url):

  csvList = []
  url = url.replace(" ","%20")
  req = Request(url)
  a = urlopen(req).read()
  soup = BeautifulSoup(a, 'html.parser')
  x = (soup.find_all('a'))

  for elem in x:
    stringElem = str(elem)

    if 'ogd' in stringElem:
      if 'csv' in stringElem:
        fileName = re.search('href="(.*)"', stringElem).group(1)
        csvList.append([fileName, extract_id(fileName)])

  df = pd.DataFrame(csvList, columns= ['fileName', 'ogd_id'])
  print(df)
  return df
  #df.to_csv('files/info.csv')

df = getInformation(url)

                                            fileName ogd_id
0  ogd35_schweizerische_elektrizitaetsbilanz_mona...     35
1                                     ogd39_epec.csv     39
2                     ogd47_vollzugsresultate_pw.csv     47
3                ogd47_vollzugsresultate_pw_2020.csv     47
4                    ogd61_vollzugsresultate_lnf.csv     61
5               ogd61_vollzugsresultate_lnf_2020.csv     61
6           ogd90_potenzialstudien_waermenutzung.csv     90


Test the adjusted files:

In [7]:
def getReport(tasks):
  for err in range(len(tasks.errors)):
    print(tasks.errors[err].message)

In [ ]:
def findIndex(path, dictList):
  for index, dictionary in enumerate(dictList):
    # if staging path is used, the
    if path in dictionary.values():
      return index

    return -1

In [6]:
staging = 'staging/'
url_ogd = 'https://www.uvek-gis.admin.ch/BFE/ogd/'
url = 'https://www.uvek-gis.admin.ch/BFE/'
dataPackage = 'datapackage.json'

for index, row in df.iterrows():

  # load original data package
  folderPath = url_ogd + row['ogd_id'] + '/'
  datapackagePath = os.path.join(folderPath, dataPackage)

  # new file path --> to staging folder
  stagingFilePath = url_ogd + 'staging/' + row['fileName']

  # store the response of URL
  response = urlopen(datapackagePath)

  # storing the JSON response
  data_json = json.loads(response.read())

  #print('original datapackage: ')
  #print(data_json)


  # change source file
  jsonAsString = str(data_json)
  jsonAsString = jsonAsString.replace(os.path.join(folderPath, row['fileName']), stagingFilePath)
  updatedJSON = ast.literal_eval(jsonAsString)

  #print('---------------------------')
  #print('adjusted datapackage: ')
  #print(updatedJSON)
  #print('===========================')

  # path for temporary datapackage
  tmpJSON_path = 'tmpPackage.json'

  # save the file as data.json
  with open(tmpJSON_path, 'w') as f:
    json.dump(updatedJSON, f)

  report = frictionless.validate(tmpJSON_path)

  if report.valid:
    #print("Valid.\n")
    k = "Everything good!"
  else:
    # check if the datapackage is written for > 1 file
    if len(report.tasks) > 1:
      j=42
      # search for correct task
    else:
      getReport(report.tasks)

  print('===========================')
  print()






Not valid!

[{'name': 'ogd47_vollzugsresultate_pw.csv',
 'type': 'table',
 'valid': False,
 'place': 'https://www.uvek-gis.admin.ch/BFE/ogd/staging/ogd47_vollzugsresultate_pw.csv',
 'labels': ['jahr',
            'nameeg',
            'nischenpool',
            'anzfz_total',
            'leergewicht',
            'co2',
            'co2_massgebend',
            'ziel',
            'ziel_abw',
            'sanktion_final',
            'gsc_beansprucht',
            'gsc_reserve',
            'anteil_lev'],
 'stats': {'errors': 2,
           'warnings': 0,
           'seconds': 0.578,
           'md5': '9e48e02e4589f98842b2bd42a1ab9eac',
           'sha256': '0cb21d5303965d149983f1895b4d3aceac2cb7fe1568256a89dd5ca597e057dc',
           'bytes': 11884,
           'fields': 13,
           'rows': 147},
 'warnings': [],
 'errors': [{'type': 'type-error',
             'title': 'Type Error',
             'description': 'The value does not match the schema type and '
                       